<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparation-of-the-proposed-region-to-train-the-Fast-RCNN" data-toc-modified-id="Preparation-of-the-proposed-region-to-train-the-Fast-RCNN-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparation of the proposed region to train the Fast RCNN</a></span></li><li><span><a href="#Region-proposal-network" data-toc-modified-id="Region-proposal-network-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Region proposal network</a></span></li><li><span><a href="#Training-RPN-network" data-toc-modified-id="Training-RPN-network-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training RPN network</a></span></li></ul></div>

This notebook aims to provide functions that produce anchor boxes as decribed in the paper.

A box will be describe either as a numpy array $[y^-, x^-, y^+, x^+]$  or as a numpy array $[c_y, c_x, h,w]$

TODO : CHECK +1

In [1]:
import numpy as np

In [195]:
def vertice_to_yxhw(anchor):
    res = (np.mean((anchor[0],anchor[2])),np.mean((anchor[1],anchor[3])), anchor[2] - anchor[0] + 1, anchor[3] - anchor[1]+1)
    return np.array(res)

In [3]:
def yxhw_to_vertice(anchor):
    res = (anchor[0] - anchor[2]/2, anchor[1] - anchor[3]/2, anchor[0] + anchor[2]/2, anchor[1] + anchor[3]/2)
    return np.array(res)

In [5]:
def anchor_box(center, ratio, scale, shape_initial, shape_featured):
    sub_width = shape_initial[0]/shape_featured[0]
    sub_height = shape_initial[1]/shape_featured[1]
    anchor_width = sub_width*scale*np.sqrt(ratio)
    anchor_height = sub_height*scale/np.sqrt(ratio)
    
    ym = center[1] - anchor_height/2
    yp = center[1] + anchor_height/2
    xm = center[0] - anchor_width/2
    xp = center[0] + anchor_width/2
    
    anchor = np.array((ym,xm,yp,xp))
    return(anchor)

In [6]:
def list_centers(shape_initial, shape_featured):
    ratio_h = shape_initial[1]/shape_featured[1]
    ratio_w = shape_initial[0]/shape_featured[0]
    #intiail center is the center at the left top corner
    all_centers = [np.array((ratio_w/2, ratio_h/2),dtype=float) + np.array((ratio_w*i, ratio_h*j),dtype=float) for i in range(int(shape_featured[0])) for j in range(int(shape_featured[1]))]
    return(all_centers)

In [7]:
def anchor_boxes(list_ratios, list_scales, shape_initial, shape_featured):
    list_center = list_centers(shape_initial, shape_featured)
    all_anchors = [anchor_box(center, ratio, scale,shape_initial,shape_featured) for center in list_center for ratio in list_ratios
                   for scale in list_scales]
    return(all_anchors)

In [8]:
def check_anchor_inside(anchor_box, shape_initial):
    ym = anchor_box[0]
    yp = anchor_box[2]
    xm = anchor_box[1]
    xp = anchor_box[3]
    is_inside = (min(xm,xp)>0) & (max(xm,xp)<shape_initial[0]) & (max(yp,ym) < shape_initial[1]) & (min(ym,yp) > 0) 
    return(is_inside)

In [9]:
def iou(box1,box2):
    xm = max(box1[1], box2[1])
    xp = min(box1[3], box2[3])
    ym = max(box1[0], box2[0])
    yp = min(box1[2], box2[2])
    
    intersection = 0
    
    if((xm < xp) &(ym < yp)):
        intersection = (xp - xm)*(yp-ym)
    
    union = (box1[3]-box1[1])*(box1[2] - box1[0]) + (box2[3]-box2[1])*(box2[2] - box2[0]) - intersection
    return(intersection/union)

In [12]:
def iou_anchors_vs_gtbox(list_anchors, list_gt_box):
    res = np.transpose([[iou(anchor, gt_box) for anchor in list_anchors] for gt_box in list_gt_box])
    return(np.array(res))

**TODO** : changer la forme de cette fonction en utilisant que des *arrays*.

In [13]:
#Return an array with :
#for all ground truth box, the anchors which maximize the IOU with it
#for all anchor, the max of the IOU

#the first column of the array is the index and the last the IOU 
def best_anchors_from_iou(dt_anchors_vs_gtbox):
    #index highest by gtbox (cond a)
    dt_anchors_vs_gtbox.argmax(axis = 0)
    ind_argmax = np.where(dt_anchors_vs_gtbox == dt_anchors_vs_gtbox.max(axis = 0))[0]
    cond_a = dt_anchors_vs_gtbox[ind_argmax,:].max(axis = 1)
    
    #highest by anchors box (cond b)
    index = dt_anchors_vs_gtbox.argmax(axis = 1)
    iou_max = dt_anchors_vs_gtbox.max(axis = 1)
    cond_b = dt_anchors_vs_gtbox[np.arange(dt_anchors_vs_gtbox.shape[0]),index]
    
    index_res = np.concatenate((ind_argmax,np.arange(dt_anchors_vs_gtbox.shape[0])))
    res = np.concatenate((cond_a, cond_b), axis=0)
    res = np.column_stack((index_res,res))
    return(res)

In [255]:
#label_from_iou returns a np.array containing for each anchor its label. (+1 if foreground, 0 if background and -1 if not used
#during the learning phase)
#The default thresholds are defined according the original paper about Fatest RCNN.

def label_from_iou(dt_anchors_vs_gtbox,pos_threshold = 0.7, neg_threshold = 0.3):
    label = np.full(dt_anchors_vs_gtbox.shape[0],-1)
    iou_max = dt_anchors_vs_gtbox.max(axis = 1)
    #positive labels : 1
    label[iou_max > pos_threshold] = 1
    #negative labels : 0
    label[iou_max < neg_threshold] = 0
    #for anchors whose maximize IOU for a given object : +1
    dt_anchors_vs_gtbox.argmax(axis = 0)
    ind_argmax = np.where(dt_anchors_vs_gtbox == dt_anchors_vs_gtbox.max(axis = 0))[0]
    label[ind_argmax] = 1
    return(label)

In [16]:
def loc(anchor_box, gt_box):
    anchor_box = vertice_to_yxhw(anchor_box)
    gt_box = vertice_to_yxhw(gt_box)
    
    y = (gt_box[0] - anchor_box[0])/anchor_box[2]
    x = (gt_box[1] - anchor_box[1])/anchor_box[3]
    w = np.log(gt_box[3]/anchor_box[3])
    h = np.log(gt_box[2]/anchor_box[2])
    
    return np.array((y,x,h,w))

In [17]:
def deloc(anchor_box, reparam_box):
    anchor_box = vertice_to_yxhw(anchor_box)
    y = anchor_box[0] + (reparam_box[0] * anchor_box[2])
    x = anchor_box[1] + (reparam_box[1] * anchor_box[3])
    h = np.exp(reparam_box[2])*anchor_box[2]
    w = np.exp(reparam_box[3])*anchor_box[3]
    return np.array((y,x,h,w))

In [240]:
def reparam_all_anchors(list_anchors, list_gt_box,pos_threshold = 0.7, neg_threshold = 0.3):
    iou = iou_anchors_vs_gtbox(list_anchors, gt_box)
    index_max_gtbox = iou.argmax(axis = 1)
    gt_box_by_anchors = [list_gt_box[i] for i in index_max_gtbox]
    #suboptimal
    res = [loc(anchor, gt_box) for anchor,gt_box in zip(list_anchors, gt_box_by_anchors)] 
    #compute labels
    labels = label_from_iou(iou, pos_threshold, neg_threshold)
    return res,labels

In [20]:
def deparam_all_anchors(list_anchors, list_box_param):
    res = [(deloc(anchor, box_param)) for anchor,box_param in zip(list_anchors, list_box_param)]
    return res

In [259]:
#TODO : heck how to fill when

def index_training_proposal(dt_anchors_vs_gtbox, nsize = 256, pos_ratio = 0.5,pos_threshold = 0.7, neg_threshold = 0.3):
    #number of positive units we need to reach in the training sample (we want a balanced sample)
    nb_pos_to_drawn = round(nsize*pos_ratio)
    lab = label_from_iou(dt_anchors_vs_gtbox, pos_threshold, neg_threshold)
    pos_lab = np.where(lab == 1)[0]
    neg_lab = np.where(lab == 0)[0]
    pos = len(pos_lab)
    neg = len(neg_lab)
    
    if (pos > nb_pos_to_drawn):
        disabled_index_pos = np.random.choice(pos_lab, size=(pos - nb_pos_to_drawn), replace = False)
        lab[disabled_index_pos] = -1
    
    if (neg > nsize - nb_pos_to_drawn):
        if(pos < nb_pos_to_drawn):
            disabled_index_neg = np.random.choice(neg_lab, size=(neg - nsize + pos), replace = False)
        else:
            disabled_index_neg = np.random.choice(neg_lab, size=(neg + nb_pos_to_drawn - nsize), replace = False)
        
        lab[disabled_index_neg] = -1    
    
    res = np.where((lab == 0) | (lab == 1))[0]
    return res

In [275]:
#TODO : heck how to fill when

def batch_training_proposal_RPN(image, feature_map, ratios, scales,gt_box,nsize = 256, pos_ratio = 0.5, pos_threshold = 0.7, neg_threshold = 0.3):
    #define all anchors using the feature map and the initial picture shapes.
    anchors_boxes = anchor_boxes(ratios, scales, image_torch.shape[2:], feature_torch.shape[2:])
    #check if each box is inside the initial image
    anchors_boxes = [box for box in anchors_boxes if check_anchor_inside(box, image_torch.shape[2:])]
    #IOU anchors vs gt box
    iou_anc_gt_box = iou_anchors_vs_gtbox(anchors_boxes, gt_box)
    #Index of the units we keep
    ind_for_sample = index_training_proposal(iou_anc_gt_box,nsize, pos_ratio)
    anchors_boxes_reparam,lab_anchors = reparam_all_anchors(anchors_boxes,gt_box,pos_threshold, neg_threshold)
    return ({"image" : image, "boxes" : torch.from_numpy(np.array(anchors_boxes_reparam)[ind_for_sample,:]),
             "labels" : torch.from_numpy(lab_anchors[ind_for_sample])})

In [261]:
image_torch = 800*torch.rand((1,3,800,800))
feature_torch = torch.rand([1,512,50,50])

ratio = [0.5, 1, 2]
anchor_scales = [8, 16, 32]

gt_box = [np.array([20, 30, 400, 500]), np.array([300, 400, 500, 600])]
labels_gt_box = np.array(("chien","chat"))

In [274]:
batch_rpn = batch_training_proposal_RPN(image_torch, feature_torch, ratio, anchor_scales, gt_box,256,0.5,0.7,0.3)

# Preparation of the proposed region to train the Fast RCNN

In [23]:
def clip_predicted_boxes(list_box, th_min, th_max):
    list_box = np.array(list_box)
    return list(np.clip(list_box,th_min,th_max))

In [24]:
#remove all boxes with at least the width or the height less that 16
def boxes_hw_min(list_box, list_score, min_size = 16):
    boxes = np.array(list_box)
    height = boxes[:, 2] - boxes[:, 0]
    width = boxes[:, 3] - boxes[:, 1]
    box_kept = np.where((height > min_size) & (width > min_size))[0]
    list_box_kept = [list_box[j] for j in box_kept]
    list_score = [list_score[j] for j in box_kept]
    return list_box_kept, list_score

In [271]:
def nms(list_box, list_score, top_pre, top_post, thresold):
    list_score = np.array(list_score)
    order = list_score.argsort()[::-1]
    order = order[:top_pre]
    keep = []
    list_box = np.array(list_box)
    
    ym = list_box[:,0]
    xm = list_box[:,1]
    yp = list_box[:,2]
    xp = list_box[:,3]
    areas = (xp - xm + 1) * (yp - ym + 1)

    while len(order)>0:
        i = order[0]
        yym = np.maximum(ym[i], ym[order[1:]])
        xxm = np.maximum(xm[i], xm[order[1:]])
        yyp = np.minimum(yp[i], yp[order[1:]])
        xxp = np.minimum(xp[i], xp[order[1:]])
        
        width = np.maximum(0.0, xxp - xxm + 1)
        height = np.maximum(0.0, yyp - yym + 1)
        intersection = width*height
        ovr = intersection/(areas[i] + areas[order[1:]] - intersection)
        
        ind_to_keep = np.where(ovr <= thresold)[0]
        order = order[ind_to_keep + 1]
        keep.append(i)
    
    keep = keep[:top_post]
    return(list_box[keep,:])

# Region proposal network

In [31]:
import torch
import torch.nn as nn
mid_channels = 512
in_channels = 512 
n_anchor = 9 #nb_weight * nb_height
conv1 = nn.Conv2d(in_channels, mid_channels, 3, 1, 1)
reg_layer = nn.Conv2d(mid_channels, n_anchor *4, 1, 1, 0)
cls_layer = nn.Conv2d(mid_channels, n_anchor *2, 1, 1, 0) 

# conv sliding layer
conv1.weight.data.normal_(0, 0.01)
conv1.bias.data.zero_()
# Regression layer
reg_layer.weight.data.normal_(0, 0.01)
reg_layer.bias.data.zero_()
# classification layer
cls_layer.weight.data.normal_(0, 0.01)
cls_layer.bias.data.zero_()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [277]:
x = torch.rand(1,512,50,50)
pred_anchor_locs = reg_layer(x)
pred_cls_scores = cls_layer(x)

In [278]:
pred_anchor_locs = pred_anchor_locs.permute(0, 2, 3, 1).contiguous().view(1, -1, 4)
print(pred_anchor_locs.shape)

pred_cls_scores = pred_cls_scores.permute(0, 2, 3, 1).contiguous()
print(pred_cls_scores)
#Out torch.Size([1, 50, 50, 18])
objectness_score = pred_cls_scores.view(1, 50, 50, 9, 2)[:, :, :, :, 1].contiguous().view(1, -1)
print(objectness_score.shape)
#Out torch.Size([1, 22500])
pred_cls_scores  = pred_cls_scores.view(1, -1, 2)
print(pred_cls_scores.shape)
# Out torch.size([1, 22500, 2])

torch.Size([1, 22500, 4])
tensor([[[[-0.1740,  0.0991,  0.0625,  ..., -0.0828, -0.1029,  0.1652],
          [-0.0816,  0.0275,  0.0383,  ..., -0.1458, -0.1004, -0.0158],
          [-0.2977,  0.0761,  0.1463,  ..., -0.2351,  0.0909,  0.0812],
          ...,
          [-0.0606, -0.0236,  0.1522,  ..., -0.1481, -0.1567,  0.0401],
          [-0.1063,  0.0574,  0.0592,  ..., -0.0924, -0.0654,  0.1167],
          [-0.1468,  0.0304,  0.1343,  ..., -0.1027, -0.1520, -0.0159]],

         [[-0.2752,  0.1217,  0.0830,  ..., -0.0507, -0.1200, -0.0832],
          [-0.1605,  0.0084,  0.2176,  ..., -0.1623, -0.0502,  0.0736],
          [-0.1656,  0.1078,  0.1727,  ..., -0.1233,  0.0354,  0.0264],
          ...,
          [-0.2993,  0.0048,  0.1715,  ..., -0.2199, -0.0642,  0.0208],
          [-0.2071,  0.0397,  0.2163,  ..., -0.1284, -0.0532, -0.0259],
          [-0.1483, -0.0049,  0.0931,  ..., -0.1115, -0.0228, -0.0831]],

         [[-0.0672, -0.0358,  0.1270,  ..., -0.1847, -0.1135,  0.0343],
    

In [203]:
#boxes as tensor [N, 5]
def roi_pooling(boxes, feature_map,scale,adaptative_max_pool):
    boxes_coord = boxes[:,1:].mul(scale).long() #scale + round
    res = [feature_map.narrow(0, boxes[i,0].int(),1)[..., boxes_coord[i,1]:(boxes_coord[i,3]+1), boxes_coord[i,0]:(boxes_coord[i,2]+1)] for i in range(boxes_coord.shape[0])]
    res = [adaptative_max_pool(element) for element in res]
    res = torch.cat(res, axis = 0)
    return(res)

In [265]:
#RPN
image = (800,800)

pred_anchor = torch.squeeze(pred_anchor_locs,0).detach().numpy()
pred_score = torch.squeeze(objectness_score,0).detach().numpy()

deparam_pred = deparam_all_anchors(list_anchors, pred_anchor)
deparam_pred = clip_predicted_boxes(deparam_pred, 0, image[1])
box_pred, score_pred = boxes_hw_min(deparam_pred, pred_score,16)
roi_pred= nms(box_pred, np.array(score_pred), 12000, 2000, 0.7)

In [199]:
#0 in labels_gt_box must be the background
def batch_training_proposal_FastRCNN(feature_map,list_box,list_gt_box,labels_gt_box, nsize = 128, pos_ratio = 0.25, pos_iou_threshold = 0.5,
                                    neg_iou_threshold_p = 0.5, neg_iou_threshold_n = 0.0, adaptative_max_pool = torch.nn.AdaptiveMaxPool2d((7,7),return_indices=False),scale = 1):
    #compute iou between each pair
    dt_anchors_vs_gtbox = iou_anchors_vs_gtbox(list_box,list_gt_box)
    
    #number of positive units we need to reach in the training sample (we want a balanced sample)
    nb_pos_to_drawn = round(nsize*pos_ratio)
    iou = iou_anchors_vs_gtbox(roi_pred, gt_box)
    #compute the maximum for each anchor
    gt_roi_label = np.argmax(iou, axis = 1)
    gt_roi_max = np.max(iou, axis = 1)
    labels = labels_gt_box[gt_roi_label]
    
    #assign the label if greater that pos_iou_threshold
    #assign background if between the two negative thresholds
    gt_pos = np.where((gt_roi_max > pos_iou_threshold))[0]
    gt_neg = np.where((gt_roi_max < neg_iou_threshold_p) & (gt_roi_max > neg_iou_threshold_n))[0] #background -- 0

    #Nb of positives and negatives boxes get using the thresholds
    pos = len(gt_pos)
    neg = len(gt_neg)
    
    #Subsampling from it
    if (pos > nb_pos_to_drawn):
        disabled_index_pos = np.random.choice(range(len(gt_pos)), size=(pos - nb_pos_to_drawn), replace = False)
        gt_pos = np.delete(gt_pos, disabled_index_pos)
    
    if (neg > nsize - nb_pos_to_drawn):
        if(pos < nb_pos_to_drawn):
            disabled_index_neg = np.random.choice(range(len(gt_neg)), size=(neg - nsize + pos), replace = False)
            gt_neg = np.delete(gt_neg, disabled_index_neg)
        else:
            disabled_index_neg = np.random.choice(range(len(gt_neg)), size=(neg + nb_pos_to_drawn - nsize), replace = False)
            gt_neg = np.delete(gt_neg, disabled_index_neg)
        
    
    #if negative : assign background labels with it's "0"
    labels[gt_neg] = "0"
    final_index = np.append(gt_pos,gt_neg)
    
    #Non reparams
    non_reparam = np.array(list_box)[final_index,:]
    #Need to transform from yxhw to xywh
    non_reparam = non_reparam[:,(1,0,3,2)]
    non_reparam = np.hstack((np.zeros((non_reparam.shape[0],1)), non_reparam))
    non_reparam = torch.from_numpy(non_reparam)
    data_for_training =roi_pooling(non_reparam, feature_map,scale, adaptive_max_pool)
    #Reparams
    reparam = [loc(box,gt_box) for box,gt_box in zip(non_reparam, list(np.array(list_gt_box)[gt_roi_label,:]))]
    
    return  data_for_training, reparam,labels[final_index]

In [204]:
data_x, box_y, label_y = batch_training_proposal_FastRCNN(test,roi_pred,gt_box,labels_gt_box, nsize = 128, pos_ratio = 0.25, pos_iou_threshold = 0.5,
                                    neg_iou_threshold_p = 0.5, neg_iou_threshold_n = 0.0,adaptative_max_pool = torch.nn.AdaptiveMaxPool2d((7,7),return_indices=False),
                                 scale = 1/16.)

In [205]:
print(data_x.shape)

torch.Size([128, 512, 4, 4])


# Training RPN network

In [279]:
import torch
import torch.nn as nn
mid_channels = 512
in_channels = 512 
n_anchor = 9 #nb_weight * nb_height
conv1 = nn.Conv2d(in_channels, mid_channels, 3, 1, 1)
reg_layer = nn.Conv2d(mid_channels, n_anchor *4, 1, 1, 0)
cls_layer = nn.Conv2d(mid_channels, n_anchor *2, 1, 1, 0) 

# conv sliding layer
conv1.weight.data.normal_(0, 0.01)
conv1.bias.data.zero_()
# Regression layer
reg_layer.weight.data.normal_(0, 0.01)
reg_layer.bias.data.zero_()
# classification layer
cls_layer.weight.data.normal_(0, 0.01)
cls_layer.bias.data.zero_()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [302]:
from torchvision import models

resnet50 = models.resnet50(pretrained=True)
image_torch = 800*torch.rand((1,3,800,800))


In [363]:
#We choose the place where we extracted the feature map in order to get H_feature * W_feature around 2400 (papers)
resnet50_features = nn.Sequential(*(list(resnet50.children())[:-5]))
resnet50_features(image_torch).shape

torch.Size([1, 256, 56, 56])

https://stackoverflow.com/questions/69480764/what-is-the-difference-between-resnet50-vgg16-etc-and-rcnn-faster-rcnn


In [367]:
class RPN(nn.Module):
    #TODO : remove embedding_dim using wv.shape[1]
    #define all the layers used in model
    def __init__(self,mid_channel, in_channels,nb_anchors,pre_trained_model):
        
        #Constructor
        super().__init__()        
        
        #embedding layer
        self.pre_trained_model = pre_trained_model
        self.conv1 = nn.Conv2d(in_channels, mid_channels, 3, 1, 1)
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv1.bias.data.zero_()
        
        self.reg_layer = nn.Conv2d(mid_channels, n_anchor *4, 1, 1, 0)
        self.reg_layer.weight.data.normal_(0, 0.01)
        self.reg_layer.bias.data.zero_()

        self.cls_layer = nn.Conv2d(mid_channels, n_anchor *2, 1, 1, 0)
        # classification layer
        self.cls_layer.weight.data.normal_(0, 0.01)
        self.cls_layer.bias.data.zero_()
       

    def forward(self,img):
        x = self.pre_trained_model(img)
        x = self.conv1(x)
        pred_anchor = self.reg_layer(x)
        pred_anchor = pred_anchor.permute(0, 2, 3, 1).contiguous().view(1, -1, 4)
        
        pred_cls = self.cls_layer(x)
        pred_cls = pred_cls.permute(0, 2, 3, 1).contiguous()
        pred_cls = pred_cls.view(1, -1, 2)

        return pred_anchor, pred_cls

In [344]:
test = torch.rand([1,36,28,28])

In [359]:
test.s

torch.Size([1, 36, 28, 28])

In [368]:
model = RPN(256,256,9,resnet50_features)
image_torch = 800*torch.rand((1,3,224,224)) #scale picture ? mean ? 
res = model(image_torch)

print(res[0].shape)
print(res[1].shape)

torch.Size([1, 28224, 4])
torch.Size([1, 28224, 2])
